## 1. Завантаження, знайомство з даними та підготовка їх до аналізу

Для початку імпортуємо необхідні бібліотеки, завантажимо датасети та уніфіюємо назви колонок у форматі snake_case для зрочної роботи.

In [3]:
import pandas as pd

In [4]:
# Завантаження даних
orders_data_path = './data/orders.csv'
supliers_data_path = './data/product-supplier.csv'

orders_df = pd.read_csv(orders_data_path)
supliers_df = pd.read_csv(supliers_data_path)

# Унфікація назв колонок
for df in [orders_df, supliers_df]:
    df.rename(columns=lambda x: x.lower().replace(" ", "_"), inplace=True)

#### 1.1 Перегляд та підготовка даних датасету orders

In [5]:
# Переглянемо як та які дані зберігаються в датасеті orders
orders_df.head()

,customer_id,customer_status,date_order_was_placed,delivery_date,order_id,product_id,quantity_ordered,total_retail_price_for_this_order,cost_price_per_unit
0,579,Silver,01-Jan-17,07-Jan-17,123002578,220101400106,2,92.6,20.70
1,7574,SILVER,01-Jan-17,05-Jan-17,123004074,210201000009,1,21.7,9.95
2,28861,Gold,01-Jan-17,04-Jan-17,123000871,230100500068,1,1.7,0.80
3,43796,Gold,01-Jan-17,06-Jan-17,123002851,220100100633,1,47.9,24.05
4,54673,Gold,01-Jan-17,04-Jan-17,123003607,220200200043,1,36.9,18.30


In [6]:
orders_df_shape = orders_df.shape
print(f"Набір даних містить {orders_df_shape[0]} рядків та {orders_df_shape[1]} колонок")

Набір даних містить 185013 рядків та 9 колонок


In [7]:
# Перевірка на пусті значення
order_missing_values = orders_df.isna().sum()
order_missing_values

customer_id                          0
customer_status                      0
date_order_was_placed                0
delivery_date                        0
order_id                             0
product_id                           0
quantity_ordered                     0
total_retail_price_for_this_order    0
cost_price_per_unit                  0
dtype: int64

Висновок: датасет не містить пустих значень.

In [8]:
# Переглянемо типи даних, що зберігаються в датасеті orders за колонками, щоб зрозуміти чи необхідно змінювати їх для правильного аналізу
orders_df.dtypes

customer_id                            int64
customer_status                       object
date_order_was_placed                 object
delivery_date                         object
order_id                               int64
product_id                             int64
quantity_ordered                       int64
total_retail_price_for_this_order    float64
cost_price_per_unit                  float64
dtype: object

Висновок: з отриманих результатів можна зробити висновок, що дані в колонці "date_order_was_placed" та "delivery_date" відображено в неправильному форматі, тож їх треба привести до типу Date.

In [9]:
# Змінемо тип даних на Date в колонках  "date_order_was_placed" та "delivery_date" 
date_columns = ["date_order_was_placed", "delivery_date"]  # Назви потрібних колонок

for col in date_columns:
    orders_df[col] = pd.to_datetime(orders_df[col], format="%d-%b-%y")

# orders_df.head()
orders_df.dtypes

customer_id                                   int64
customer_status                              object
date_order_was_placed                datetime64[ns]
delivery_date                        datetime64[ns]
order_id                                      int64
product_id                                    int64
quantity_ordered                              int64
total_retail_price_for_this_order           float64
cost_price_per_unit                         float64
dtype: object

In [13]:
# Також вважаю за потрібне уніфікувати значення в колонці customer_status 
# та конвертувати все до нижнього регістру, щоб повсюди мати однаковий формат 
# і було зручно працювати з ними.

orders_df.customer_status = orders_df.customer_status.apply(lambda value: value.lower())
orders_df.head()

,customer_id,customer_status,date_order_was_placed,delivery_date,order_id,product_id,quantity_ordered,total_retail_price_for_this_order,cost_price_per_unit
0,579,silver,2017-01-01,2017-01-07,123002578,220101400106,2,92.6,20.70
1,7574,silver,2017-01-01,2017-01-05,123004074,210201000009,1,21.7,9.95
2,28861,gold,2017-01-01,2017-01-04,123000871,230100500068,1,1.7,0.80
3,43796,gold,2017-01-01,2017-01-06,123002851,220100100633,1,47.9,24.05
4,54673,gold,2017-01-01,2017-01-04,123003607,220200200043,1,36.9,18.30


#### 1.2 Перегляд та підготовка даних датасету product-supplier

In [25]:
# Переглянемо як та які дані зберігаються в датасеті product-supplier
supliers_df.head()

,product_id,product_line,product_category,product_group,product_name,supplier_country,supplier_name,supplier_id
0,210100100001,Children,Children Outdoors,"Outdoor things, Kids",Boy's and Girl's Ski Pants with Braces,NO,Scandinavian Clothing A/S,50
1,210100100002,Children,Children Outdoors,"Outdoor things, Kids",Children's Jacket,ES,Luna sastreria S.A.,4742
2,210100100003,Children,Children Outdoors,"Outdoor things, Kids",Children's Jacket Sidney,NO,Scandinavian Clothing A/S,50
3,210100100004,Children,Children Outdoors,"Outdoor things, Kids",Children's Rain Set,NO,Scandinavian Clothing A/S,50
4,210100100005,Children,Children Outdoors,"Outdoor things, Kids",Children's Rain Suit,NO,Scandinavian Clothing A/S,50


In [26]:
supliers_df_shape = supliers_df.shape
print(f"Набір даних містить {supliers_df_shape[0]} рядків та {supliers_df_shape[1]} колонок")

Набір даних містить 5504 рядків та 8 колонок


In [43]:
# Перевірка на пусті значення
supliers_missing_values = supliers_df.isna().sum()
supliers_missing_values

product_id          0
product_line        0
product_category    0
product_group       0
product_name        0
supplier_country    0
supplier_name       0
supplier_id         0
dtype: int64

Висновок: датасет не містить пустих значень

In [27]:
# Переглянемо типи даних, що зберігаються в датасеті orders за колонками, щоб зрозуміти чи необхідно змінювати їх для правильного аналізу
supliers_df.dtypes

product_id           int64
product_line        object
product_category    object
product_group       object
product_name        object
supplier_country    object
supplier_name       object
supplier_id          int64
dtype: object

Висновок: з отриманих даних можна сказати, що дані містять правильні типи тож немає необхідності зміни типів для даного датасету.